In [1]:
import mesa
import random
import pickle
import networkx as nx
import seaborn as sns
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
class MaleAdult(mesa.Agent):

    def __init__(self, unique_id, model):
        # Pass the parameters to the parent class.
        super().__init__(unique_id, model)

        self.infected = False
        self.days = 0
        self.antibody = 0
        
    def step(self):
        if self.antibody > 0:
            self.antibody -= 1
            if self.antibody == 0:
               print(f"I'm agent {str(self.unique_id)} in {str(self.pos)} and I can be infected again") 
            else:
                print(f"I'm agent {str(self.unique_id)} in {str(self.pos)} and I currently have antibodies for {str(self.antibody)} more days") 
        
        if self.infected:
            a = random.randint(1,100)
            if a <= self.model.cont_rate:
                other_agent = self.random.choice(self.model.grid.get_cell_list_contents([self.pos]))
                if other_agent is not None and other_agent.infected == False and other_agent.antibody == 0:
                    print(f"I'm agent {str(self.unique_id)} in {str(self.pos)} and I spread the infection to {str(other_agent.unique_id)} which is in {str(other_agent.pos)}")
                    other_agent.infected = True
                    other_agent.days = 5
                    self.model.num_infected += 1
            
            print(f"I'm agent {str(self.unique_id)} in {str(self.pos)} and i'm infected, let's see if I die")
            b = random.randint(1,100)
            if b <= self.model.death_rate:
                print(f"I'm agent {str(self.unique_id)} in {str(self.pos)} and I'm dead")
                self.model.kill_agents.append(self)
            else:
                self.days -= 1
                if self.days == 0:
                    print(f"I'm agent {str(self.unique_id)} in {str(self.pos)} and I'm cured")
                    self.infected = False
                    self.model.num_infected -= 1
                    self.antibody = self.model.antibody_days
                    
        self.move()
                    
    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            include_center=False)
        new_position = self.random.choice(possible_steps)
        distance = self.model.G[self.pos][new_position]["KM_TOT"]
        pop = len(self.model.G.nodes[new_position]['agent'])
        prob_mov = 10 + pop/(4*distance)
        r = random.randint(1, 100)
        if prob_mov >= r:
            print(f"I'm agent {str(self.unique_id)} and I moved")
            print("Actual position: " + self.pos)
            print("New position: " + new_position)
            print("Value of r: " + str(r))
            print("Probability of moving: " + str(prob_mov))
            print("Distance: " + str(distance) + " Population: " + str(pop))
            self.model.grid.move_agent(self, new_position)
        
        
                

In [3]:
with open('distances.gpickle', 'rb') as f:
    G = pickle.load(f)

In [4]:
class NetworkInfectionModel(mesa.Model):

    def __init__(self, G, death_rate=5, recovery_days=5, antibody_days=10, cont_rate=20):
        super().__init__()
        self.recovery_days = recovery_days
        self.antibody_days = antibody_days
        self.death_rate = death_rate
        self.cont_rate = cont_rate
        self.num_infected = 0
        self.num_dead = 0
        self.total_agents = 0

        self.G = G
        self.grid = mesa.space.NetworkGrid(self.G)

        self.schedule = mesa.time.RandomActivation(self)
        self.kill_agents = []

        # Create agents
        for i, node in enumerate(self.G.nodes(data = True)):
            node_dict = node[1]
            population = next(iter(node_dict.values()))
            print(node[0] + " will have "+ str(round(population/1000)) + " agents")
            for x in range(round(population/1000)):
               a = MaleAdult(i+1, self)
               self.schedule.add(a) 
               self.total_agents += 1
               self.grid.place_agent(a, node[0])
               infected = np.random.choice([0,1], p=[0.99,0.01])
               if infected == 1:
                   a.infected = True
                   self.num_infected += 1
                   a.days = recovery_days

    def step(self):
        self.schedule.step()
        for x in self.kill_agents[:]:
            self.grid.remove_agent(x)
            self.schedule.remove(x)
            self.kill_agents.remove(x)
            self.num_infected -= 1 
            self.num_dead += 1
            self.total_agents -= 1
        print(f"Current number of total agents is: {str(self.total_agents)}")
        print(f"Current number of infected agents is: {str(self.num_infected)}")
        print(f"Current number of dead agents is: {str(self.num_dead)}")
        for node in enumerate(self.G.nodes(data = True)):
           print(node[1][0] + ": " + str(len(node[1][1]['agent'])))
            

In [5]:
model = NetworkInfectionModel(G, cont_rate= 50)

Alessano will have 6 agents
Gagliano del Capo will have 5 agents
Corsano will have 6 agents
Tiggiano will have 3 agents
Tricase will have 18 agents
Salve will have 5 agents
Specchia will have 5 agents
Castrignano del Capo will have 5 agents
Patù will have 2 agents
Presicce-Acquarica will have 10 agents
Morciano di Leuca will have 3 agents
Miggiano will have 4 agents
Montesano Salentino will have 3 agents
Taurisano will have 13 agents
Andrano will have 5 agents
Ruffano will have 10 agents
Ugento will have 12 agents
Surano will have 2 agents
Spongano will have 4 agents
Nociglia will have 2 agents
Diso will have 3 agents
San Cassiano will have 2 agents
Melissano will have 7 agents
Botrugno will have 3 agents
Casarano will have 20 agents
Racale will have 11 agents
Ortelle will have 2 agents
Taviano will have 12 agents
Supersano will have 5 agents
Alliste will have 7 agents
Castro will have 2 agents
Poggiardo will have 6 agents
Scorrano will have 7 agents
Muro Leccese will have 5 agents
San

In [6]:
x = 1
while model.num_infected > 0:
    print(f"Day {str(x)}:")
    model.step()
    x+= 1

Day 1:
I'm agent 802 and I moved
Actual position: Guagnano
New position: Copertino
Value of r: 2
Probability of moving: 10.29126213592233
Distance: 20.6 Population: 24
I'm agent 790 and I moved
Actual position: Salice Salentino
New position: Lizzanello
Value of r: 6
Probability of moving: 10.090361445783133
Distance: 33.2 Population: 12
I'm agent 544 and I moved
Actual position: Vernole
New position: Gagliano del Capo
Value of r: 7
Probability of moving: 10.020325203252032
Distance: 61.5 Population: 5
I'm agent 77 and I moved
Actual position: Taurisano
New position: Sanarica
Value of r: 1
Probability of moving: 10.02
Distance: 25.0 Population: 2
I'm agent 560 in Lecce and I spread the infection to 572 which is in Lecce
I'm agent 560 in Lecce and i'm infected, let's see if I die
I'm agent 558 and I moved
Actual position: Lecce
New position: Casarano
Value of r: 6
Probability of moving: 10.103092783505154
Distance: 48.5 Population: 20
I'm agent 111 and I moved
Actual position: Ugento
New

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




Value of r: 2
Probability of moving: 10.081967213114755
Distance: 18.3 Population: 6
I'm agent 170 and I moved
Actual position: Campi Salentina
New position: Aradeo
Value of r: 3
Probability of moving: 10.010330578512397
Distance: 48.4 Population: 2
I'm agent 274 in Neviano and i'm infected, let's see if I die
I'm agent 502 and I moved
Actual position: Lequile
New position: Taurisano
Value of r: 4
Probability of moving: 10.044326241134751
Distance: 56.4 Population: 10
I'm agent 506 in Sannicola and i'm infected, let's see if I die
I'm agent 216 in Sternatia and I currently have antibodies for 7 more days
I'm agent 531 and I moved
Actual position: Sanarica
New position: Guagnano
Value of r: 10
Probability of moving: 10.025167785234899
Distance: 59.6 Population: 6
I'm agent 16 and I moved
Actual position: Leverano
New position: Lizzanello
Value of r: 4
Probability of moving: 10.028735632183908
Distance: 26.1 Population: 3
I'm agent 270 in Sternatia and I spread the infection to 60 which